周内效应——右侧买入

In [ ]:
import pandas as pd
from datetime import datetime
import trdb2py
import numpy as np

isStaticImg = False
width = 960
height = 768

pd.options.display.max_columns = None
pd.options.display.max_rows = None

trdb2cfg = trdb2py.loadConfig('./trdb2.yaml')

In [ ]:
# 具体基金
# asset = 'jrj.510310'
# baselineasset = 'jrj.510310'
asset = 'jqdata.000300_XSHG|1d'

# 起始时间，0表示从最开始算起
tsStart = 0
tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))

# 结束时间，-1表示到现在为止
tsEnd = -1
tsEnd = int(trdb2py.str2timestamp('2020-09-30', '%Y-%m-%d'))

# 初始资金池
paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

# 买入参数，用全部的钱来买入（也就是复利）
paramsbuy = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
)

paramsbuy1 = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
    nextTimes=1,
)

# 买入参数，用全部的钱来买入（也就是复利）
paramsbuy2 = trdb2py.trading2_pb2.BuyParams(
    moneyParts=2,
)

# 卖出参数，全部卖出
paramssell = trdb2py.trading2_pb2.SellParams(
    perVolume=1,
)

# 卖出参数，全部卖出
paramssell7 = trdb2py.trading2_pb2.SellParams(
#     perVolume=1,
    keepTime=7 * 24 * 60 * 60,
)

lststart = [1, 2, 3, 4, 5]
lsttitle = ['周一', '周二', '周三', '周四', '周五']

In [ ]:
def calcweekday2val2(wday, offday):
    if offday == 1:
        if wday == 5:
            return 3
    if offday == 2:
        if wday >= 4:
            return 4
    if offday == 3:
        if wday >= 3:
            return 5
    if offday == 4:
        if wday >= 2:
            return 6
    if offday == 5:
        if wday >= 1:
            return 7        
        
    return offday


大家好，我是格子衫小C（微信公众号：格子衫小C），我们今天继续研究周内效应。

上次和大家聊到了左侧买入，今天接着这个话题聊聊右侧买入。

右侧买入就是在信号出现后，再买入。

我们依然基于国内场外基金交易策略探讨右侧交易的可能。

目标策略如下：

1. 如果今天是周四，且当日收盘价在 ema 29 日线上方，则买入，并于下周周三无条件卖出。
2. 如果今天是周一，且当日收盘价在 ema 29 日线下方，则买入，并于下周周五无条件卖出。
3. 如果周一已经买入，则周四不操作，反之亦然。

交易规则如下：

1. 交易日下午3点以前买入，按当日价格结算份额。
2. 交易日下午3点以后买入，则按下一个交易日价格结算（周五下午3点后买入，按周一价格结算，QDII基金一般还要再延后）。

In [ ]:
# baseline    
s0 = trdb2py.trading2_pb2.Strategy(
    name="normal",
    asset=trdb2py.str2asset(asset),         
)
        
buy0 = trdb2py.trading2_pb2.CtrlCondition(
    name='buyandhold',
)

# paramsbuy = trdb2py.trading2_pb2.BuyParams(
#     perHandMoney=1,
# )

# paramsinit = trdb2py.trading2_pb2.InitParams(
#     money=10000,
# )

s0.buy.extend([buy0])
s0.paramsBuy.CopyFrom(paramsbuy)
s0.paramsInit.CopyFrom(paramsinit)        
p0 = trdb2py.trading2_pb2.SimTradingParams(
    assets=[trdb2py.str2asset(asset)],
    startTs=tsStart,
    endTs=tsEnd,
    strategies=[s0],
    title='沪深300',
)  

pnlBaseline = trdb2py.simTrading(trdb2cfg, p0)
trdb2py.showPNL(pnlBaseline, toImg=isStaticImg, width=width, height=height)

In [ ]:
lstparams = []

lstassetcode = ['jqdata.000300_XSHG|1d']
lstassetname = ['沪深300 目标策略']

# tsStart = 0
# tsEnd = -1

for ai in range(0, len(lstassetcode)):
    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(lstassetcode[ai]),         
    )

    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday2',
        vals=[4, calcweekday2val2(4, 4)],
    )

    buy1 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['up'],
        strVals=['ema.{}'.format(29)],
    )

    buy2 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday2',
        vals=[1, calcweekday2val2(1, 4)],
        group=1,
    )

    buy3 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['down'],
        strVals=['ema.{}'.format(29)],
        group=1,    
    )

    sell0 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday',
        vals=[3],
    )

    sell1 = trdb2py.trading2_pb2.CtrlCondition(
        name='ctrlconditionid',
        vals=[1],
        strVals=['buy'],
    )

    sell2 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday',
        vals=[5],
        group=1,     
    )

    sell3 = trdb2py.trading2_pb2.CtrlCondition(
        name='ctrlconditionid',
        vals=[2],
        strVals=['buy'],
        group=1,     
    )

#     paramsbuy = trdb2py.trading2_pb2.BuyParams(
#         perHandMoney=1,
#     )

#     paramsinit = trdb2py.trading2_pb2.InitParams(
#         money=10000,
#     )
    
    feebuy = trdb2py.trading2_pb2.FeeParams(
        percentage=0.0003,
    )
    
    feesell = trdb2py.trading2_pb2.FeeParams(
        percentage=0.0003,
    )    

    s0.buy.extend([buy0, buy1, buy2, buy3])
    s0.sell.extend([sell0, sell1, sell2, sell3])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell) 
    s0.paramsInit.CopyFrom(paramsinit)        
#     s0.feeBuy.CopyFrom(feebuy)        
#     s0.feeSell.CopyFrom(feesell)            
    p0 = trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(lstassetcode[ai])],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title=lstassetname[ai],
    )  

    lstparams.append(p0)

lstpnl1 = trdb2py.simTradings(trdb2cfg, lstparams)

trdb2py.showPNLs(lstpnl1 + [pnlBaseline], toImg=isStaticImg, width=width, height=height)

In [ ]:
dfpnl = trdb2py.buildPNLReport([pnlBaseline] + lstpnl1)

# dfpnl1 = dfpnl[dfpnl['totalReturns'] >= 1]

dfpnl[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility', 'variance']].sort_values(by='totalReturns', ascending=False)

上面是目标策略和基线PNL曲线，我们直接将交易规则改为右侧交易，也就是周四得到交易信号，但周五再交易，看看结果如何（其实不用看也知道结果一定不好）。

In [ ]:
lstparams = []

lstassetcode = ['jqdata.000300_XSHG|1d']
lstassetname = ['沪深300 右侧买入目标策略']

# tsStart = 0
# tsEnd = -1

for ai in range(0, len(lstassetcode)):
    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(lstassetcode[ai]),         
    )

    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday2',
        vals=[4, calcweekday2val2(4, 4)],
    )

    buy1 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['up'],
        strVals=['ema.{}'.format(29)],
    )

    buy2 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday2',
        vals=[1, calcweekday2val2(1, 4)],
        group=1,
    )

    buy3 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['down'],
        strVals=['ema.{}'.format(29)],
        group=1,    
    )

    sell0 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday',
        vals=[3],
    )

    sell1 = trdb2py.trading2_pb2.CtrlCondition(
        name='ctrlconditionid',
        vals=[1],
        strVals=['buy'],
    )

    sell2 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday',
        vals=[5],
        group=1,     
    )

    sell3 = trdb2py.trading2_pb2.CtrlCondition(
        name='ctrlconditionid',
        vals=[2],
        strVals=['buy'],
        group=1,     
    )

#     paramsbuy = trdb2py.trading2_pb2.BuyParams(
#         perHandMoney=1,
#         nextTimes=1,
#     )

#     paramsinit = trdb2py.trading2_pb2.InitParams(
#         money=10000,
#     )
    
    feebuy = trdb2py.trading2_pb2.FeeParams(
        percentage=0.0003,
    )
    
    feesell = trdb2py.trading2_pb2.FeeParams(
        percentage=0.0003,
    )    

    s0.buy.extend([buy0, buy1, buy2, buy3])
    s0.sell.extend([sell0, sell1, sell2, sell3])
    s0.paramsBuy.CopyFrom(paramsbuy1)
    s0.paramsSell.CopyFrom(paramssell) 
    s0.paramsInit.CopyFrom(paramsinit)        
#     s0.feeBuy.CopyFrom(feebuy)        
#     s0.feeSell.CopyFrom(feesell)            
    p0 = trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(lstassetcode[ai])],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title=lstassetname[ai],
    )  

    lstparams.append(p0)

lstpnl2 = trdb2py.simTradings(trdb2cfg, lstparams)

trdb2py.showPNLs(lstpnl1 + lstpnl2 + [pnlBaseline], toImg=isStaticImg, width=width, height=height)

In [ ]:
dfpnl = trdb2py.buildPNLReport([pnlBaseline] + lstpnl1 + lstpnl2)

# dfpnl1 = dfpnl[dfpnl['totalReturns'] >= 1]

dfpnl[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility', 'variance']].sort_values(by='totalReturns', ascending=False)

结果当然是非常之不好。

我们的目标策略是基于周内效应来做的，前面得出的结论是：

1. 周四下跌的概率最大。
2. 周一的波动率最大，上涨趋势里，涨幅大，下跌趋势里，跌幅大。

因此目标策略本质是在上涨趋势里，避过周四；在下跌趋势里，避过周一。

如果我们改为右侧买入（只有买入需要考虑ema，卖出只考虑周几），就是将周四买入变成了周五买入，这能好才怪呢。

顺便插一句，为啥A股市场会有周内效应呢？

我和一些炒股的朋友聊天，发现很多朋友虽然没有仔细研究过周内效应，但他们都有周一价格波动最大，周三周四价格会相对稳定下来的概念。
他们的理解是，周一因为要消化周末2天的消息，所以波动最大，然后经过周二的平复，周三周四相对来说，是最好的交易时机。
A股应该也是因为参与玩家大都是类似这个想法，所以才会形成这样的局面吧。

但就数据来看，美股和港股也都是一周交易5天，周末休息2天，但他们表现却没这么明显。

接下来，我们实测一下，如果右侧买入，应该用怎样的参数最合适。

我们遍历5-60日ema均线，周一到周五，得到信号后，第二天买入，分别持有2-5天（因为第二天才买入，所以持有时间需要延长1天），一共55x5X4=22000种情况。

In [ ]:
lstparams = []

for ema in range(5, 61):    
    for sdo in range(2, 6):
        for sd in range(1, 6):
            buy0 = trdb2py.trading2_pb2.CtrlCondition(
                name='weekday2',
                vals=[sd, calcweekday2val2(sd, sdo)],
            )

            buy1 = trdb2py.trading2_pb2.CtrlCondition(
                name='indicatorsp',
                operators=['up'],
                strVals=['ema.{}'.format(ema)],
            )
            
            sell0 = trdb2py.trading2_pb2.CtrlCondition(
                name='weekday',
                vals=[trdb2py.nextWeekDay(sd, sdo)],
            )

            sell1 = trdb2py.trading2_pb2.CtrlCondition(
                name='ctrlconditionid',
                vals=[1],
                strVals=['buy'],
            )            
            
            for edo in range(1, 5):
                for ed in range(1, 6):

                    buy2 = trdb2py.trading2_pb2.CtrlCondition(
                        name='weekday2',
                        vals=[ed, calcweekday2val2(ed, edo)],
                        group=1,
                    )

                    buy3 = trdb2py.trading2_pb2.CtrlCondition(
                        name='indicatorsp',
                        operators=['down'],
                        strVals=['ema.{}'.format(ema)],
                        group=1,    
                    )

                    sell2 = trdb2py.trading2_pb2.CtrlCondition(
                        name='weekday',
                        vals=[trdb2py.nextWeekDay(ed, edo)],
                        group=1,     
                    )

                    sell3 = trdb2py.trading2_pb2.CtrlCondition(
                        name='ctrlconditionid',
                        vals=[2],
                        strVals=['buy'],
                        group=1,     
                    )
            

                    s0 = trdb2py.trading2_pb2.Strategy(
                        name="normal",
                        asset=trdb2py.str2asset(asset),
                    )

                    s0.buy.extend([buy0, buy1, buy2, buy3])
                    s0.sell.extend([sell0, sell1, sell2, sell3])
                    s0.paramsBuy.CopyFrom(paramsbuy1)
                    s0.paramsSell.CopyFrom(paramssell) 
                    s0.paramsInit.CopyFrom(paramsinit)        
                    lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
                        assets=[trdb2py.str2asset(asset)],
                        startTs=tsStart,
                        endTs=tsEnd,
                        strategies=[s0],
                        title='ema{} up{}持有{}天 down{}持有{}天 右侧'.format(ema, lsttitle[sd-1], sdo, lsttitle[ed-1], edo),
                    ))
 
lstpnlmix = trdb2py.simTradings(trdb2cfg, lstparams, ignoreTotalReturn=6)

trdb2py.showPNLs2(lstpnlmix + [pnlBaseline] + lstpnl1, baseline=pnlBaseline, showNums=5, toImg=isStaticImg, width=width, height=height)

In [ ]:
dfpnl = trdb2py.buildPNLReport(lstpnlmix + [pnlBaseline] + lstpnl1)

# dfpnl1 = dfpnl[dfpnl['totalReturns'] >= 1]

dfpnl[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility', 'variance']].sort_values(by='totalReturns', ascending=False)

虽然有策略和目标策略差别不大，但我们看到是周三持有4天，也就是周四买入，持有3天，而周五持有2天，就是周一买入，持有1天。

这个策略和前面得到的结果有差异，这个参数，避开的日期太多了。

In [ ]:
# 起始时间，0表示从最开始算起
tsStart = 0
# tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))

# 结束时间，-1表示到现在为止
tsEnd = -1
# tsEnd = int(trdb2py.str2timestamp('2020-09-30', '%Y-%m-%d'))

# baseline    
s0 = trdb2py.trading2_pb2.Strategy(
    name="normal",
    asset=trdb2py.str2asset(asset),         
)
        
buy0 = trdb2py.trading2_pb2.CtrlCondition(
    name='buyandhold',
)

# paramsbuy = trdb2py.trading2_pb2.BuyParams(
#     perHandMoney=1,
# )

# paramsinit = trdb2py.trading2_pb2.InitParams(
#     money=10000,
# )

s0.buy.extend([buy0])
s0.paramsBuy.CopyFrom(paramsbuy)
s0.paramsInit.CopyFrom(paramsinit)        
p0 = trdb2py.trading2_pb2.SimTradingParams(
    assets=[trdb2py.str2asset(asset)],
    startTs=tsStart,
    endTs=tsEnd,
    strategies=[s0],
    title='沪深300',
)  

pnlBaselineF = trdb2py.simTrading(trdb2cfg, p0)
trdb2py.showPNL(pnlBaselineF, toImg=isStaticImg, width=width, height=height)

In [ ]:
lstparams = []

lstassetcode = ['jqdata.000300_XSHG|1d']
lstassetname = ['沪深300 目标策略']

tsStart = 0
tsEnd = -1

for ai in range(0, len(lstassetcode)):
    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(lstassetcode[ai]),         
    )

    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday2',
        vals=[4, calcweekday2val2(4, 4)],
    )

    buy1 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['up'],
        strVals=['ema.{}'.format(29)],
    )

    buy2 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday2',
        vals=[1, calcweekday2val2(1, 4)],
        group=1,
    )

    buy3 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['down'],
        strVals=['ema.{}'.format(29)],
        group=1,    
    )

    sell0 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday',
        vals=[3],
    )

    sell1 = trdb2py.trading2_pb2.CtrlCondition(
        name='ctrlconditionid',
        vals=[1],
        strVals=['buy'],
    )

    sell2 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday',
        vals=[5],
        group=1,     
    )

    sell3 = trdb2py.trading2_pb2.CtrlCondition(
        name='ctrlconditionid',
        vals=[2],
        strVals=['buy'],
        group=1,     
    )

#     paramsbuy = trdb2py.trading2_pb2.BuyParams(
#         perHandMoney=1,
#     )

#     paramsinit = trdb2py.trading2_pb2.InitParams(
#         money=10000,
#     )
    
    feebuy = trdb2py.trading2_pb2.FeeParams(
        percentage=0.0003,
    )
    
    feesell = trdb2py.trading2_pb2.FeeParams(
        percentage=0.0003,
    )    

    s0.buy.extend([buy0, buy1, buy2, buy3])
    s0.sell.extend([sell0, sell1, sell2, sell3])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell) 
    s0.paramsInit.CopyFrom(paramsinit)        
#     s0.feeBuy.CopyFrom(feebuy)        
#     s0.feeSell.CopyFrom(feesell)            
    p0 = trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(lstassetcode[ai])],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title=lstassetname[ai],
    )  

    lstparams.append(p0)

lstpnl1F = trdb2py.simTradings(trdb2cfg, lstparams)

trdb2py.showPNLs(lstpnl1F + [pnlBaselineF], toImg=isStaticImg, width=width, height=height)

In [ ]:
lstparams = []

lstassetcode = ['jqdata.000300_XSHG|1d']
lstassetname = ['沪深300 ema26右侧']

# 起始时间，0表示从最开始算起
tsStart = 0
# tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))

# 结束时间，-1表示到现在为止
tsEnd = -1
# tsEnd = int(trdb2py.str2timestamp('2020-09-30', '%Y-%m-%d'))

for ai in range(0, len(lstassetcode)):
    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(lstassetcode[ai]),         
    )

    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday2',
        vals=[3, calcweekday2val2(3, 4)],
    )

    buy1 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['up'],
        strVals=['ema.{}'.format(26)],
    )

    buy2 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday2',
        vals=[5, calcweekday2val2(5, 2)],
        group=1,
    )

    buy3 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['down'],
        strVals=['ema.{}'.format(26)],
        group=1,    
    )

    sell0 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday',
        vals=[2],
    )

    sell1 = trdb2py.trading2_pb2.CtrlCondition(
        name='ctrlconditionid',
        vals=[1],
        strVals=['buy'],
    )

    sell2 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday',
        vals=[2],
        group=1,     
    )

    sell3 = trdb2py.trading2_pb2.CtrlCondition(
        name='ctrlconditionid',
        vals=[2],
        strVals=['buy'],
        group=1,     
    )

#     paramsbuy = trdb2py.trading2_pb2.BuyParams(
#         perHandMoney=1,
#         nextTimes=1,
#     )

#     paramsinit = trdb2py.trading2_pb2.InitParams(
#         money=10000,
#     )
    
    feebuy = trdb2py.trading2_pb2.FeeParams(
        percentage=0.0003,
    )
    
    feesell = trdb2py.trading2_pb2.FeeParams(
        percentage=0.0003,
    )    

    s0.buy.extend([buy0, buy1, buy2, buy3])
    s0.sell.extend([sell0, sell1, sell2, sell3])
    s0.paramsBuy.CopyFrom(paramsbuy1)
    s0.paramsSell.CopyFrom(paramssell) 
    s0.paramsInit.CopyFrom(paramsinit)        
#     s0.feeBuy.CopyFrom(feebuy)        
#     s0.feeSell.CopyFrom(feesell)            
    p0 = trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(lstassetcode[ai])],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title=lstassetname[ai],
    )  

    lstparams.append(p0)

lstpnl2t = trdb2py.simTradings(trdb2cfg, lstparams)

trdb2py.showPNLs(lstpnl2t + lstpnl1F + [pnlBaselineF], toImg=isStaticImg, width=width, height=height)

我们把回测数据放开，就能看到问题了。

虽然在测试数据区间内，咱们得到了一个看似有效的参数，但放入非测试数据里，会发现情况并不理想。
这就是回测过拟合的问题。

这也是为什么前面，我们会拿这一组参数还测过十多种指数，都能说明问题，才一直使用这套参数的原因。

这里也不是说右侧交易就没办法，只能说，ema指标不适合目标策略的右侧日间交易（当然，也有可能是我们测试参数的宽度不够）。

最后，还要强调一下过拟合，有些同学应该能想到，咱们可以拿完整数据去回测，说不定会得到一个在完整数据上都能表现得非常不错的参数。

但这个项目本身，就过拟合的，在历史数据上，得到一条完美的PNL曲线是不难的，但我们的目的难道是这个么？我们不是要找到一个能适用于未来数据的参数么？（至少也得是未来一段时间都适用的）

In [ ]:
lstema = [26, 24, 27, 28, 25, 23, 24, 26, 28, 15]
lstday1 = [3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
lstod1 = [4, 4, 4, 4, 4, 4, 5, 5, 2, 5]
lstday2 = [5, 5, 5, 5, 5, 5, 5, 5, 4, 4]
lstod2 = [2, 2, 2, 2, 2, 2, 2, 2, 4, 3]
lstparams = []

# 起始时间，0表示从最开始算起
tsStart = 0
# tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))

# 结束时间，-1表示到现在为止
tsEnd = -1
# tsEnd = int(trdb2py.str2timestamp('2020-09-30', '%Y-%m-%d'))

for ii in range(0, len(lstema)):
    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday2',
        vals=[lstday1[ii], calcweekday2val2(lstday1[ii], lstod1[ii])],
    )

    buy1 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['up'],
        strVals=['ema.{}'.format(lstema[ii])],
    )

    sell0 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday',
        vals=[trdb2py.nextWeekDay(lstday1[ii], lstod1[ii])],
    )

    sell1 = trdb2py.trading2_pb2.CtrlCondition(
        name='ctrlconditionid',
        vals=[1],
        strVals=['buy'],
    )            
            
    buy2 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday2',
        vals=[lstday2[ii], calcweekday2val2(lstday2[ii], lstod2[ii])],
        group=1,
    )

    buy3 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['down'],
        strVals=['ema.{}'.format(lstema[ii])],
        group=1,    
    )

    sell2 = trdb2py.trading2_pb2.CtrlCondition(
        name='weekday',
        vals=[trdb2py.nextWeekDay(lstday2[ii], lstod2[ii])],
        group=1,     
    )

    sell3 = trdb2py.trading2_pb2.CtrlCondition(
        name='ctrlconditionid',
        vals=[2],
        strVals=['buy'],
        group=1,     
    )


    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(asset),
    )

    s0.buy.extend([buy0, buy1, buy2, buy3])
    s0.sell.extend([sell0, sell1, sell2, sell3])
    s0.paramsBuy.CopyFrom(paramsbuy1)
    s0.paramsSell.CopyFrom(paramssell) 
    s0.paramsInit.CopyFrom(paramsinit)        
    lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(asset)],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title='ema{} up{}持有{}天 down{}持有{}天 右侧'.format(lstema[ii], lsttitle[lstday1[ii]-1], lstod1[ii], lsttitle[lstday2[ii]-1], lstod2[ii]),
    ))
 
lstpnlmixF = trdb2py.simTradings(trdb2cfg, lstparams)

trdb2py.showPNLs2(lstpnlmixF + lstpnl1F + [pnlBaselineF], toImg=isStaticImg, width=width, height=height)

In [ ]:
dfpnl = trdb2py.buildPNLReport(lstpnlmixF + lstpnl1F + [pnlBaselineF])

# dfpnl1 = dfpnl[dfpnl['totalReturns'] >= 1]

dfpnl[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility', 'variance']].sort_values(by='totalReturns', ascending=False)